In [2]:
import pandas as pd
import numpy as np

In [3]:
students = pd.read_csv("Data/Students data/students_processed.csv")
batla = pd.read_csv("Output/BONUS_HASAM_202206235.csv")

C:\Users\yoave\AppData\Local\Temp\ipykernel_12492\3675610163.py:1: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  students = pd.read_csv("Data/Students data/students_processed.csv")


In [4]:
df = pd.merge(batla, students, on="id", how="left")

In [5]:
df = df.drop(columns=["end_date_1","end_date_2","end_date_3","end_date_4","end_date_5","end_date_6"])

In [6]:
df["date"] =  pd.to_datetime(df["date"])
df["end_date"] =  pd.to_datetime(df["end_date"])

def inter(x):

    if x["date"].year == x["end_date"].year and x["date"].month == x["end_date"].month:
        return 1
    else:
        return 0

df["intervention"] = df.apply(inter, axis=1)

In [7]:
## After merge with students

### Index monthes before and after intervention
df = df.sort_values(by=['id','date'], ascending=False)
df['intervention_index_pre'] = df.groupby('id',group_keys=False)['intervention'].apply(lambda x: (x - x.cumsum()).where(x == 1, lambda x: x.cumsum()))

df = df.sort_values(by=['id','date'], ascending=True)
df['intervention_index_post'] = df.groupby('id',group_keys=False)['intervention'].apply(lambda x: (x - x.cumsum()).where(x == 1, lambda x: x.cumsum()))
df["intervention_index_post"] = df["intervention_index_post"]*-1

def intervention(x):
    if x["intervention_index_pre"] < 0:
        return x["intervention_index_pre"]
    if x["intervention_index_pre"] == 0:
        return x["intervention_index_post"]
    else:
        return 0

df["intervention_index"] = df.apply(intervention, axis=1)
df= df.drop(columns=["intervention_index_pre","intervention_index_post","intervention"])

In [14]:
df = df.loc[(df["end_date"]>="2000-01-01")]

In [15]:
df.to_csv("Output/merge_stud_batla.csv", index=False)